In [22]:
import csv
import os
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat

# Generate RSA private/public key pair
rsa_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
rsa_public_key = rsa_private_key.public_key()

# Generate ECDSA private/public key pair
ec_private_key = ec.generate_private_key(ec.SECP256R1())
ec_public_key = ec_private_key.public_key()

# Write the public keys to files
with open('rsa_public_key.pem', 'wb') as f:
    f.write(rsa_public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo))
with open('ec_public_key.pem', 'wb') as f:
    f.write(ec_public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo))

# Serialize the private key to PEM format and save it to a file
with open('rsa_private_key.pem', 'wb') as f:
    f.write(rsa_private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))
with open('ecdsa_private_key.pem', 'wb') as f:
    f.write(ec_private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Load the CSV file
with open('data.csv', 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

# Encrypt the CSV file using RSA
encrypted_data = []
for row in data:
    encrypted_row = []
    for cell in row:
        encrypted_cell = rsa_public_key.encrypt(
            bytes(cell, 'utf-8'),
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        encrypted_row.append(encrypted_cell.hex())
    encrypted_data.append(encrypted_row)

# Sign the encrypted CSV file using ECDSA
signature = ec_private_key.sign(
    bytes(str(encrypted_data), 'utf-8'),
    ec.ECDSA(hashes.SHA256())
)

# Write the encrypted data and signature to files
with open('encrypted_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(encrypted_data)

with open('signature.dat', 'wb') as f:
    f.write(signature)

In [23]:
import pandas as pd

In [24]:
df_encrypted = pd.read_csv('encrypted_data.csv', header=None)

In [34]:
df_encrypted.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,22f1d2cbbf027ea70903777276cfac30140925f28cf4a3...,8abe3f67357a2ad9604e774c0004a8fa84221b9dcce0f1...,01946be5eff5c619bdf3e4305ecfc590472d124f7acd32...,bd8f29a8d5c5e3975735f47a545d86d37ccc8499cf29b5...,9010c6c7865c82a5893b91662814df3494dd1d8555a868...,81cc9d0f7c40c4c84a81c550f09512cd51fff9009460a4...,07ac6da425c34d7e2dc1b24036f2ba77d75694baece1fd...,432100181b53a53194e5c30de4a16ca1dd690a70b68c0a...,d1b050486cdba7324249957b8a17f9427dab7d818fc8e1...,70284430b81c623e9c32c1ad130aac39ba5aedd379b0d7...,...,05bb9e82cae67823e94fa66ce121e3baabb036d9159384...,bd26d008cb6a50314b1134ada65103303a1a21aea2afc5...,86d6ec1bffea2884a186301dd04343c9921e2556b1497f...,159d635a6de92f4d173afb7dfdec7b98939c35bc7c6436...,c438ec07f1f8cc4b3e4fb935d3936844aef4908fd5a7bb...,12e07ad264cfe80d7cb4527eb0612a139eb292d0f5bb17...,5577572a8ad2696ef235b29dabd1682ff613b89a91c564...,32272de7409d04b887699f01b0ec6a9c4a8957e574f44d...,590d88aa77b4658b5d98ea9ebc30b0dd50d1aa79c2376e...,ce822962b601392c058a0251be47d44d7030f945516014...
1,331e23921f3a74fb372a31d6ff7455d9e1fc7b13a30b90...,62b17c42a0c922639834566ad0dc8ef6a0f7e1481a7e9b...,0b92bfd5570c78be9bcff1119188e8d7de2cd8b65d9601...,9d3ee047740c375d9eed998932568dfd5a6462899b1f60...,b0fe28748a6b9c5f2af22db93fc47f6a3234208101cb02...,07ce2d1aa078b5e5bb73256a6333df462e2aefb6c042d1...,6665661bd540b7576989b692bcfa6316d637bc702ed018...,45894c766f26c953fcdfe5bdd37deaa2437e83030400b9...,9280f7ad23221a5f7ff0028734e3cc2bf573d0ef27225f...,5add3d8c04864cf44dda2145e0846c6d9fde73c8682aca...,...,29d418fc0423b831f8e6fc5b805531b532f6b2c027dc74...,9adb391446efe9b3d620c884776bc6b6e6300570f0a72d...,51845241cbd4ba9e7aa3b4832f71ba6a7e18452dd0913f...,0cc2a7273a93d0187fc993396c7b51f58a7d5027482b7e...,86a663da568e6e52f27c423fe9d1b62dd79aa243b78a8f...,527c96888cbdd16decacf57669e3cc0b2a7320f0b7ed33...,92a0f695df8d9ea512a79a8756ba1cb7319de7742ad1ae...,61c0c9c8d5da8b28abbb0cbfa60b715ab9f5ffe8d8bed3...,6a5259d48cdc63fd47cc3b1a468c9077d2df5abe440e75...,3a1ba0fc00bc3ae36b008604e633b53a7fd344f5d582a9...
2,9e5d0cfe98e27b7e5231e138b4ed0e9f33d7a851bc63fc...,cc45e3fbf38f9f140f467963bb7e649b3f989fbef27483...,3bb6ec83f252e1b3d85888d81ffea47d4dae4971beccc8...,97db3c0f49bd068b73c96db2aacc70dea7fb905cabd489...,60a7ebaaa044cc07c09ceff645ffbea2c3c3c847fc3fea...,2032d7f307c0546b4858c3c969e0b48fabd7d726458c50...,557215b56db7a62626f484fd202925c94253400638513a...,03070032e1f2745bd16d9aec83805b6ef44b503f76ce07...,0fe98df567468df600145cbe91fe7a60cd29816e7e7762...,67583bfb7c1c289775aba1d374784b977ce17e09a56619...,...,98284ca209be9dfc0837090da9b6de6b9de8e335231781...,110fa30b34e67dbd49dcb1bc77e4b6818fc7e8bcfffa8e...,209f2ddbadc509eb2c4c9525abe7c0b19e6cc0afd53170...,536ea5d5cac899d377a8c13d497121935a06aa9fc98660...,2c81971925d2688b4a1bc8fdec8afea731ebfe9e60925e...,3aaa1fbc96d7010de00ccd639df6448b40cdcac3eb775a...,95b23527078ad575bd9c7f831a6d896dd103eb28904150...,bd6e44b2b5b00a0f48f8a43a92a9b30705791c1aa2a5d4...,5f6a0c6ffd93a984d1892c7b3322092ab0e5e420932891...,38a822dd5df2bf96e16d017807662f703f5b7005a3aa76...
3,1846d8f1eae3d9fcefb8264fbcb933e2e833a478a68e9c...,835de0ecbe3cb6e843e2ff8676c947a6a917c1e2a25b69...,856746c0ad75842cbefea8fe30e289c46e7279aef52e02...,07e0e2f9579d0374c2941b9d71b00b5186c4ac8fbeedc3...,c9b8545f9f45b1fd735b02891fe29ee0c9551a7ed349b2...,cd94f7d97dbfb8408c927569bbed51fa1a6622bcb25941...,08a76e6668aafe43dca82ddcba106a4468f10f78191a88...,0f234f4b4f03ee5f8ea333551c452d74cc5c29dafabee9...,31821fdff2ff935fa9cc0e48a27a41870e9841ad28cd42...,559ba11c25536d0d40b50bdf8527a6ba88c4008964b707...,...,984537433ee659d7584180feea02ea55afd710358efda1...,c9190aa3e66debee7643460389739cc1c5f6c55b943a92...,7a936627cbd0581bcabe8fad653fbb0f81d98de2f93525...,5fa3753cdb37bb25cbdff9d3454e2f59afbc2fd73cb4c6...,b005b26963b1388fdbf9dc512fd0fe892f4d2b9954044e...,1edfef3d3fd27819674158372772dd4619bf1c72fdfe32...,1b4dea28da06fb2dd219fecd302aec4d95144d5603a709...,200b4716c01830c11032f9f6c83ba962cf4268ddf75570...,3ec4fae8fc538dcee93ef

In [26]:
# Load the private keys from files
with open('rsa_private_key.pem', 'rb') as f:
    rsa_private_key = serialization.load_pem_private_key(f.read(), password=None)
with open('ecdsa_private_key.pem', 'rb') as f:
    ec_private_key = serialization.load_pem_private_key(f.read(), password=None)

# Load the public keys from files
with open('rsa_public_key.pem', 'rb') as f:
    rsa_public_key = serialization.load_pem_public_key(f.read())
with open('ec_public_key.pem', 'rb') as f:
    ec_public_key = serialization.load_pem_public_key(f.read())

# Load the encrypted data from the CSV file
with open('encrypted_data.csv', 'r') as f:
    reader = csv.reader(f)
    encrypted_data = list(reader)

# Decrypt the encrypted data using RSA
#decrypted_data = []
#for encrypted_row in encrypted_data:
#    decrypted_row = []
#    for encrypted_cell in encrypted_row:
#        decrypted_cell = rsa_private_key.decrypt(
#            bytes.fromhex(encrypted_cell),
#            padding.OAEP(
#                mgf=padding.MGF1(algorithm=hashes.SHA256()),
#                algorithm=hashes.SHA256(),
#                label=None
#            )
#       ).decode('utf-8')
#        decrypted_row.append(decrypted_cell)
#    decrypted_data.append(decrypted_row)

# Decrypt the encrypted data and store it in a new DataFrame
# Create an empty DataFrame to store the decrypted data
df_decrypted = pd.DataFrame(columns=df_encrypted.columns)

# Loop over each column in the input DataFrame
for column in df_encrypted.columns:
    # Decrypt the data in the column and store it in the new DataFrame
    decrypted_column = []
    for index, row in df_encrypted.iterrows():
        encrypted_message = bytes.fromhex(row[column])
        decrypted_message = rsa_private_key.decrypt(
            encrypted_message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        decrypted_column.append(decrypted_message.decode('utf-8'))
    df_decrypted[column] = decrypted_column



# Verify the signature using ECDSA
with open('signature.dat', 'rb') as f:
    signature = f.read()

try:
    ec_public_key.verify(
        signature,
        bytes(str(encrypted_data), 'utf-8'),
        ec.ECDSA(hashes.SHA256())
    )
    print('Signature is valid')
except InvalidSignature:
    print('Signature is invalid')

Signature is valid


In [27]:
df_orignial = pd.read_csv('data.csv', header=None)

In [33]:
df_orignial.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,2,11,2013,58.00,75.0,65.0,0.08,67.0,69.00,...,338.0,369.0,318.00,322.0,345.0,298.00,316.00,299.0,341.00,304.0
1,0,3,11,2013,0.16,0.1,68.0,69.00,75.0,0.09,...,67.0,74.0,61.00,84.0,74.0,67.00,77.00,91.0,0.07,61.0
2,0,4,11,2013,0.08,66.0,77.0,64.00,84.0,164.00,...,114.0,106.0,128.00,511.0,74.0,84.00,59.00,69.0,69.00,71.0
3,0,5,11,2013,68.00,58.0,95.0,81.00,74.0,0.06,...,167.0,202.0,0.53,226.0,256.0,315.00,421.00,66.0,0.09,64.0
4,0,6,11,2013,77.00,61.0,97.0,65.00,78.0,66.00,...,117.0,345.0,351.00,69.0,57.0,0.07,0.07,72.0,56.00,78.0


In [32]:
df_decrypted.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,2,11,2013,58.0,75.0,65.0,0.08,67.0,69.0,...,338.0,369.0,318.0,322.0,345.0,298.0,316.0,299.0,341.0,304.0
1,0,3,11,2013,0.16,0.1,68.0,69.0,75.0,0.09,...,67.0,74.0,61.0,84.0,74.0,67.0,77.0,91.0,0.07,61.0
2,0,4,11,2013,0.08,66.0,77.0,64.0,84.0,164.0,...,114.0,106.0,128.0,511.0,74.0,84.0,59.0,69.0,69.0,71.0
3,0,5,11,2013,68.0,58.0,95.0,81.0,74.0,0.06,...,167.0,202.0,0.53,226.0,256.0,315.0,421.0,66.0,0.09,64.0
4,0,6,11,2013,77.0,61.0,97.0,65.0,78.0,66.0,...,117.0,345.0,351.0,69.0,57.0,0.07,0.07,72.0,56.0,78.0


In [38]:
decrypted_data

[['0',
  '2',
  '11',
  '2013',
  '58.0',
  '75.0',
  '65.0',
  '0.08',
  '67.0',
  '69.0',
  '0.07',
  '73.0',
  '68.0',
  '0.06',
  '77.0',
  '69.0',
  '91.0',
  '64.0',
  '74.0',
  '74.0',
  '74.0',
  '67.0',
  '61.0',
  '89.0',
  '58.0',
  '75.0',
  '57.0',
  '108.0',
  '85.0',
  '89.0',
  '183.0',
  '272.0',
  '311.0',
  '0.37',
  '498.0',
  '274.0',
  '276.0',
  '0.23',
  '187.0',
  '136.0',
  '79.0',
  '42.0',
  '91.0',
  '148.0',
  '819.0',
  '798.0',
  '802.0',
  '738.0',
  '508.0',
  '253.0',
  '84.0',
  '68.0',
  '65.0',
  '611.0',
  '162.0',
  '167.0',
  '183.0',
  '0.73',
  '711.0',
  '695.0',
  '601.0',
  '439.0',
  '211.0',
  '93.0',
  '162.0',
  '0.4',
  '518.0',
  '484.0',
  '0.58',
  '795.0',
  '494.0',
  '616.0',
  '672.0',
  '611.0',
  '345.0',
  '415.0',
  '675.0',
  '612.0',
  '353.0',
  '445.0',
  '409.0',
  '0.36',
  '356.0',
  '332.0',
  '344.0',
  '321.0',
  '328.0',
  '407.0',
  '482.0',
  '0.46',
  '338.0',
  '369.0',
  '318.0',
  '322.0',
  '345.0',
  '298.